In [ ]:
here::i_am("readme.md")
library(here)
library(dplyr)

here() starts at C:/Users/Plancha/Desktop/PACD


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




O nosso objetivo é de prever o número de *sets* que um jogo de ténis profissional de Austrália vai ter, baseado num número de recursos que um jogo vai ter. 

Um jogo de ténis é composto por pontos, jogos e sets. De acordo com as [regras de ténis da Federação Internacional de Tennis (IFT)](https://www.atptour.com/en/corporate/rulebook), um set é uma pontuação, onde o primeiro jogador a ganhar 6 jogos ganha um set, sendo um jogo decidido pelo número de sets que um jogador ganha. Um jogo pode ser à melhor de 3 ou à melhor de 5, ou seja, o jogador precisa de ganhar 2 sets (o que traduz para 3 sets de máximo) ou 3 sets (máximo 5), respectivamente.  O jogador que decide o lado que começa/quem serve primeiro é decidido aleatoriamente, sendo que o outro decide a outra decisão.

Baseado nestes factos, parece que o projeto vai ser de regressão ordinal.

In [ ]:
arrow::read_parquet(here("data", "atpRaw.parquet"), as_data_frame = TRUE) -> atp
head(atp)

Born,Date,GameRank,GameRound,Ground,Hand,Height,LinkPlayer,Location,Oponent,PlayerName,Prize,Score,Tournament,WL,_id.$oid
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Belgrade, Serbia",2021.11.22 - 2021.12.05,30,Semi-Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Madrid, Spain",Marin Cilic,Novak Djokovic,,64 62,Davis Cup Finals,W,624ab34913b144c54b3c9aba
"Belgrade, Serbia",2022.02.21 - 2022.02.26,26,Round of 16,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Dubai, U.A.E.",Karen Khachanov,Novak Djokovic,"$2,794,840",63 76,Dubai,W,624ab34913b144c54b3c9abb
"Belgrade, Serbia",2021.11.15 - 2021.11.21,12,Round Robin,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Turin, Italy",Cameron Norrie,Novak Djokovic,"$7,250,000",62 61,Nitto ATP Finals,W,624ab34913b144c54b3c9abc
"Belgrade, Serbia",2021.11.15 - 2021.11.21,5,Round Robin,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Turin, Italy",Andrey Rublev,Novak Djokovic,"$7,250,000",63 62,Nitto ATP Finals,W,624ab34913b144c54b3c9abd
"Belgrade, Serbia",2021.11.01 - 2021.11.07,2,Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Paris, France",Daniil Medvedev,Novak Djokovic,"�5,207,405",46 63 63,ATP Masters 1000 Paris,W,624ab34913b144c54b3c9abe
"Belgrade, Serbia",2021.11.01 - 2021.11.07,10,Semi-Finals,Hard,"Right-Handed, Two-Handed Backhand",188,https://www.atptour.com/en/players/novak-djokovic/d643/player-activity?year=all&matchType=Singles,"Paris, France",Hubert Hurkacz,Novak Djokovic,"�5,207,405",36 60 76,ATP Masters 1000 Paris,W,624ab34913b144c54b3c9abf


In [ ]:
nrow(atp)

[1] 1308835

A base de dados mostra informação pública de [ATP Tour](https://www.atptour.com/en/players).

A primeira vista dos dados revela vários factos:

* A base de dados tem 1308835, sendo cada registro um jogo de um jogador "principal" (com as informações de tal) contra um oponente (sem as informações de tal).
    * Isto poderá indicar que teremos de transformar os dados em registros de jogos entre os dois jogadores, sem jogador principal. Não temos a certeza se isto é necessário.
* Ao fazer a cabeça dos dados, parece que apenas mostra o jogador Novak Djokovic como principal. Isto pode indicar que a base de dados pode estar ordenada de uma certa forma.
    * Esta ordem apenas pode indicar como os dados foram obtidos, e nada sobre um registro específico. Informação sobre a recolha pode ajudar a perceber melhor o contexto da base de dados.
* A

In [ ]:
# temp
library(tidygeocoder)
?tidygeocoder

tidygeocoder-package {tidygeocoder},R Documentation


In [ ]:
atp %>% distinct(PlayerName)

PlayerName
<chr>
Novak Djokovic
Daniil Medvedev
Rafael Nadal
Alexander Zverev
Stefanos Tsitsipas
Matteo Berrettini
Andrey Rublev
Casper Ruud
Felix Auger-Aliassime
